In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
N_centr = 1
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,100,0,20,20,40,40,60,60,80,80,100]
pt_boarders = [0.5,0.9,0.2,0.5,0.5,1.0,1.0,5.0,0.5,5]
file_path="../VTXalignment/input/"
file_names=["my-m_ee_Run14AuAu_158th_new_20086_900runs.root"]
legend_names = ["pos","neg","sim_pos","sim_neg"] 
hist_select_3D_names = ["rich_prob1","rich_prob2","rich_prob3","disp_hist_el"]

In [3]:
N_hists = len(hist_select_3D_names)

In [4]:
hists_read, hists_sim = [], []

for iFile in range(len(file_names)):
    #print(file_path+file_names[iFile])
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_read_file = []
    for ihist in range(N_hists):
        hists_read_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_read_file[-1].SetDirectory(root.nullptr)
        hists_read_file[-1].SetName(hists_read_file[-1].GetName()+f"{iFile}")
    hists_read.append(hists_read_file)

infile.Close()

In [5]:
type_loc = 1
iFile = 0
c0 = root.TCanvas(f"c0",f"c0",2200,1400)
c0.Divide(3,2)
projz = hists_read[iFile][type_loc].ProjectionZ(f"psdfsrojz{type_loc}2")
legends = []
eid_bins = [[1,1],[2,3],[7,8],[8,9],[4,6],[10,12]]
hist2ds = []
for iid in range(len(eid_bins)):
    c0.cd(1+iid)
    ibinz0 = projz.FindBin(eid_bins[iid][0]-0.5)
    ibinz1 = projz.FindBin(eid_bins[iid][1]-0.5)
    hist2d = root.TH2D(f"new_hist2d_{iid}",f"new_hist2d_{iid}",hists_read[iFile][type_loc].GetXaxis().GetNbins(),hists_read[iFile][type_loc].GetXaxis().GetXmin(),hists_read[iFile][type_loc].GetXaxis().GetXmax(),
                       hists_read[iFile][type_loc].GetYaxis().GetNbins(),hists_read[iFile][type_loc].GetYaxis().GetXmin(),hists_read[iFile][type_loc].GetYaxis().GetXmax())
    hist2ds.append(hist2d)
    for ibin_x in range(1,hists_read[iFile][type_loc].GetXaxis().GetNbins()+1):
        for ibin_y in range(1,hists_read[iFile][type_loc].GetYaxis().GetNbins()+1):
            content = 0
            for ibin_z in range(ibinz0,ibinz1+1):
                content += hists_read[iFile][type_loc].GetBinContent(ibin_x,ibin_y,ibin_z)
            hist2d.SetBinContent(ibin_x,ibin_y,content)
    

    hist2d.GetXaxis().SetNdivisions(505)
    hist2d.Draw("colz")
    legends.append(root.TLegend(0.5,0.65,0.99,0.95,f"{iid}"))
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/m2_tofe.png")

In [6]:
type_loc = 1
iFile = 0
c0 = root.TCanvas(f"c0",f"c0",1400,1400)
c0.Divide(2,2)
projz = hists_read[iFile][type_loc].ProjectionZ(f"psdfsrojz{type_loc}2")
legends = []
eid_bins = [[1,1],[2,3],[7,8],[9,9]]
eid_refs = [[4,6],[4,6],[10,12],[10,12]]
hist2ds, hist2ds_refs = [], []
for iid in range(len(eid_bins)):
    c0.cd(1+iid)
    ibinz0 = projz.FindBin(eid_bins[iid][0]-0.5)
    ibinz1 = projz.FindBin(eid_bins[iid][1]-0.5)
    ibinz0_ref = projz.FindBin(eid_refs[iid][0]-0.5)
    ibinz1_ref = projz.FindBin(eid_refs[iid][1]-0.5)
    hist2d = root.TH2D(f"new_hist2d_{iid}",f"new_hist2d_{iid}",hists_read[iFile][type_loc].GetXaxis().GetNbins(),hists_read[iFile][type_loc].GetXaxis().GetXmin(),hists_read[iFile][type_loc].GetXaxis().GetXmax(),
                       hists_read[iFile][type_loc].GetYaxis().GetNbins(),hists_read[iFile][type_loc].GetYaxis().GetXmin(),hists_read[iFile][type_loc].GetYaxis().GetXmax())
    hist2ds_ref = root.TH2D(f"new_hist2d_ref_{iid}",f"new_hist2d_ref_{iid}",hists_read[iFile][type_loc].GetXaxis().GetNbins(),hists_read[iFile][type_loc].GetXaxis().GetXmin(),hists_read[iFile][type_loc].GetXaxis().GetXmax(),
                       hists_read[iFile][type_loc].GetYaxis().GetNbins(),hists_read[iFile][type_loc].GetYaxis().GetXmin(),hists_read[iFile][type_loc].GetYaxis().GetXmax())
    hist2ds.append(hist2d)
    hist2ds_refs.append(hist2ds_ref)
    for ibin_x in range(1,hists_read[iFile][type_loc].GetXaxis().GetNbins()+1):
        for ibin_y in range(1,hists_read[iFile][type_loc].GetYaxis().GetNbins()+1):
            content = 0
            for ibin_z in range(ibinz0,ibinz1+1):
                content += hists_read[iFile][type_loc].GetBinContent(ibin_x,ibin_y,ibin_z)
            if content>10: hist2ds[-1].SetBinContent(ibin_x,ibin_y,content)
            else: hist2ds[-1].SetBinContent(ibin_x,ibin_y,0)
            for ibin_z in range(ibinz0_ref,ibinz1_ref+1):
                content += hists_read[iFile][type_loc].GetBinContent(ibin_x,ibin_y,ibin_z)
            hist2ds_refs[-1].SetBinContent(ibin_x,ibin_y,content)

    hist2ds[-1].Divide(hist2ds_refs[-1])
    hist2ds[-1].GetXaxis().SetNdivisions(505)
    hist2ds[-1].Draw("colz")
    legends.append(root.TLegend(0.5,0.65,0.99,0.95,f"{iid}"))
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/m2_tofe.png")

In [7]:
type_loc = 2
iFile = 0
c0 = root.TCanvas(f"c0",f"c0",1400,1400)
c0.Divide(2,2)
projz = hists_read[iFile][type_loc].ProjectionZ(f"psdfsrojz{type_loc}2")
legends = []
eid_bins = [[1,1],[2,3],[7,8],[9,9]]
eid_refs = [[4,6],[4,6],[10,12],[10,12]]
hist2ds, hist2ds_refs = [], []
for iid in range(len(eid_bins)):
    c0.cd(1+iid)
    ibinz0 = projz.FindBin(eid_bins[iid][0]-0.5)
    ibinz1 = projz.FindBin(eid_bins[iid][1]-0.5)
    ibinz0_ref = projz.FindBin(eid_refs[iid][0]-0.5)
    ibinz1_ref = projz.FindBin(eid_refs[iid][1]-0.5)
    hist2d = root.TH2D(f"new_hist2d_{iid}",f"new_hist2d_{iid}",hists_read[iFile][type_loc].GetXaxis().GetNbins(),hists_read[iFile][type_loc].GetXaxis().GetXmin(),hists_read[iFile][type_loc].GetXaxis().GetXmax(),
                       hists_read[iFile][type_loc].GetYaxis().GetNbins(),hists_read[iFile][type_loc].GetYaxis().GetXmin(),hists_read[iFile][type_loc].GetYaxis().GetXmax())
    hist2ds_ref = root.TH2D(f"new_hist2d_ref_{iid}",f"new_hist2d_ref_{iid}",hists_read[iFile][type_loc].GetXaxis().GetNbins(),hists_read[iFile][type_loc].GetXaxis().GetXmin(),hists_read[iFile][type_loc].GetXaxis().GetXmax(),
                       hists_read[iFile][type_loc].GetYaxis().GetNbins(),hists_read[iFile][type_loc].GetYaxis().GetXmin(),hists_read[iFile][type_loc].GetYaxis().GetXmax())
    hist2ds.append(hist2d)
    hist2ds_refs.append(hist2ds_ref)
    for ibin_x in range(1,hists_read[iFile][type_loc].GetXaxis().GetNbins()+1):
        for ibin_y in range(1,hists_read[iFile][type_loc].GetYaxis().GetNbins()+1):
            content = 0
            for ibin_z in range(ibinz0,ibinz1+1):
                content += hists_read[iFile][type_loc].GetBinContent(ibin_x,ibin_y,ibin_z)
            if content>10: hist2ds[-1].SetBinContent(ibin_x,ibin_y,content)
            else: hist2ds[-1].SetBinContent(ibin_x,ibin_y,0)
            for ibin_z in range(ibinz0_ref,ibinz1_ref+1):
                content += hists_read[iFile][type_loc].GetBinContent(ibin_x,ibin_y,ibin_z)
            hist2ds_refs[-1].SetBinContent(ibin_x,ibin_y,content)

    hist2ds[-1].Divide(hist2ds_refs[-1])
    hist2ds[-1].GetXaxis().SetNdivisions(505)
    hist2ds[-1].Draw("colz")
    legends.append(root.TLegend(0.5,0.65,0.99,0.95,f"{iid}"))
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/m2_tofe.png")